<a href="https://colab.research.google.com/github/zaku2590/signate/blob/main/%E3%82%B2%E3%83%BC%E3%83%A0%E3%81%AE%E5%8B%9D%E6%95%97%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
sample_submit = pd.read_csv("sample_submit.csv", index_col=0, header=None)

train = train.drop(["blueTotalGold"], axis=1)
test = test.drop(["blueTotalGold"], axis=1)

train["blueKills-blueDeaths"] = train["blueFirstBlood"] + train["blueKills"] - train["blueDeaths"]
test["blueKills-blueDeaths"] = test["blueFirstBlood"] + test["blueKills"] - test["blueDeaths"]

train = train.drop(["blueFirstBlood"], axis=1)
test = test.drop(["blueFirstBlood"], axis=1)

train["blueEliteMonsters+blueDragons"] = train["blueEliteMonsters"] + train["blueDragons"]
test["blueEliteMonsters+blueDragons"] = test["blueEliteMonsters"] + test["blueDragons"]

train["con"] = train["blueKills-blueDeaths"] * train["blueAssists"]
test["con"] = test["blueKills-blueDeaths"] * test["blueAssists"]

先頭5行表示

In [33]:
train.head(5)

,blueKills,blueDeaths,blueAssists,blueTotalExperience,blueWins,blueKills-blueDeaths,blueEliteMonsters+blueDragons
gameId,,,,,,,
0,5,8,6,17256,0,-3,0
1,10,1,5,17863,0,10,0
2,3,10,2,17256,0,-7,0
3,7,10,8,18201,0,-2,0
4,4,9,4,17256,0,-5,0


In [18]:
test.head()

,blueFirstBlood,blueAssists,blueTotalExperience,blueKills-blueDeaths,blueEliteMonsters-blueDragons
gameId,,,,,
9,0,6,18201,1,0
15,0,6,18021,0,1
18,1,3,17256,2,0
23,0,7,17256,1,0
31,0,9,18472,2,0


In [ ]:
print(train.shape)
print(test.shape)

(8000, 9)
(2000, 8)


欠損の確認

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 0 to 9998
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   blueFirstBlood        8000 non-null   int64
 1   blueAssists           8000 non-null   int64
 2   blueEliteMonsters     8000 non-null   int64
 3   blueDragons           8000 non-null   int64
 4   blueTotalExperience   8000 non-null   int64
 5   blueWins              8000 non-null   int64
 6   blueKills-blueDeaths  8000 non-null   int64
dtypes: int64(7)
memory usage: 500.0 KB


In [11]:
train['blueWins'].value_counts()

,count
blueWins,
0,4066
1,3934


統計値

In [19]:
blue_wins_data = train[train["blueWins"] == 1]
blue_wins_data.describe()

,blueFirstBlood,blueAssists,blueTotalExperience,blueWins,blueKills-blueDeaths,blueEliteMonsters-blueDragons
count,3934.000000,3934.000000,3934.000000,3934.0,3934.000000,3934.000000
mean,0.628114,6.892476,18241.174377,1.0,1.645399,0.232334
std,0.483370,3.191630,643.779242,0.0,3.579127,0.427163
min,0.000000,0.000000,16650.000000,1.0,-11.000000,-1.000000
25%,0.000000,5.000000,18021.000000,1.0,0.000000,0.000000
50%,1.000000,7.000000,18201.000000,1.0,2.000000,0.000000
75%,1.000000,9.000000,18491.000000,1.0,4.000000,0.000000
max,1.000000,17.000000,20101.000000,1.0,13.000000,2.000000


In [38]:
train.describe()

,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalExperience,blueWins,blueKills-blueDeaths,blueEliteMonsters+blueDragons
count,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,6.029750,5.92350,5.881875,0.538625,0.358125,17983.514000,0.491750,0.619250,0.896750
std,2.623876,2.64892,3.189897,0.629728,0.479479,772.348402,0.499963,4.107206,1.049632
min,0.000000,0.00000,0.000000,0.000000,0.000000,16650.000000,0.000000,-13.000000,0.000000
25%,4.000000,4.00000,3.000000,0.000000,0.000000,17256.000000,0.000000,-2.000000,0.000000
50%,6.000000,6.00000,6.000000,0.000000,0.000000,18021.000000,0.000000,1.000000,0.000000
75%,8.000000,8.00000,8.000000,1.000000,1.000000,18472.000000,1.000000,3.000000,2.000000
max,14.000000,14.00000,17.000000,2.000000,1.000000,20101.000000,1.000000,14.000000,3.000000


ブルーの勝利との相関係数

In [42]:
train.corrwith(train["blueWins"])

,0
blueKills,0.337541
blueDeaths,-0.237224
blueAssists,0.311647
blueEliteMonsters,0.272810
blueDragons,0.251957
blueTotalExperience,0.328167
blueWins,1.000000
blueKills-blueDeaths,0.396204
blueEliteMonsters+blueDragons,0.278769
con,0.360596


In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# 目的変数（y）と説明変数（X）の分離
y = train["blueWins"]  # 青チームが勝った場合1、それ以外は0
X = train.drop(["blueWins"], axis=1)  # blueWins以外の特徴量

# パイプライン作成（標準化 + ロジスティック回帰）
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 特徴量を標準化
    ('logreg', LogisticRegression(solver='lbfgs', max_iter=500))  # ロジスティック回帰モデル
])

# モデル学習
pipeline.fit(X, y)

# テストデータで予測（青チームが勝つ確率）
pred_proba = pipeline.predict_proba(test)[:, 1]  # ポジティブクラスの確率（青チームが勝つ確率）

# しきい値0.5を基準に0または1に変換
pred_binary = (pred_proba >= 0.5).astype(int)

# 提出用データの作成
# sample_submit["blueWins"] = pred_proba  # 確率で提出する場合
sample_submit = pd.DataFrame(index=test.index)
sample_submit["blueWins"] = pred_binary  # 0または1で提出する場合

# CSVに保存
sample_submit.to_csv("submission.csv", index=True, header=False)  # 提出用ファイル保存

# 結果の確認
print("予測確率 (最初の5件):", pred_proba[:5])  # 確率の確認
print("予測結果 (最初の5件):", pred_binary[:5])  # 0または1の確認

予測確率 (最初の5件): [0.45156688 0.55904897 0.38216014 0.33330633 0.57966499]
予測結果 (最初の5件): [0 1 0 0 1]
